In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import time
from tqdm import tqdm
from unidecode import unidecode

In [2]:
tqdm.pandas()

### Read

In [124]:
#Reads the excel file as a dict, joining all worksheets to one
df_dict = pd.read_excel("C:/Users/User/Documents/data_sets/indicadoressegurancapublicamunic.xlsx", sheet_name=None) 
# Data source https://dados.mj.gov.br/dataset/sistema-nacional-de-estatisticas-de-seguranca-publica

'\nFONTES DE DADOS E INDICADORES\n\nOs dados disponíveis foram extraídos das soluções SinespJC e Sinesp Integração, fontes primárias dos seguintes indicadores: Totais de Ocorrências e Totais de Vítimas de estupro, furto de veículos, homicídio doloso, lesão corporal seguida de morte, roubo à instituição financeira, roubo de carga, roubo de veículos e roubo seguido de morte.\n\nNOTA\n\nÉ importante ressaltar que as informações apresentadas refletem o nível de alimentação e consolidação de cada Unidade da Federação no SinespJC e Sinesp Integração na data de sua extração, podendo ocorrer atualizações posteriores à publicação. Salientamos que se considera como último período os dados consolidados que antecedem os últimos três meses, por exemplo: Em fev/2019, serão publicados os dados de jan/2015 a out/2018; em mar/2019 os dados de jan/2015 a nov/2018; e assim sucessivamente. Isso se faz necessário para que os Gestores Estaduais possam coletar, tratar e validar os dados antes do fornecimento

### Compile

In [125]:
#Concates all the dicts readed before into one single pd.DataFrame()
df = pd.concat(df_dict.values(), ignore_index=True)
#The excel file above, is a file with multiple sheets. 
#This step joins all the sheets into one single DataFrame

### Data cleaning

In [126]:
#Take a look at the dataframe
df.head(5)

,Cód_IBGE,Município,Sigla UF,Região,Mês/Ano,Vítimas
0,1200138,Bujari,AC,NORTE,2018-01-01,1
1,1200203,Cruzeiro do Sul,AC,NORTE,2018-01-01,8
2,1200302,Feijó,AC,NORTE,2018-01-01,3
3,1200328,Jordão,AC,NORTE,2018-01-01,1
4,1200336,Mâncio Lima,AC,NORTE,2018-01-01,1


In [127]:
#Renames all columns_names to a pattern
df = df.rename(columns={'Município':'Municipio', 'Sigla UF': 'Sigla_UF', 'Mês/Ano':'Mes_Ano', 'Vítimas':'Vitimas', 'Região': 'Regiao'}).copy()
#Column Cód_IBGE won't be needed on our analysis, so it can be discarded for memory usage improvement
df = df.drop('Cód_IBGE', axis=1)

In [128]:
df.head(5)

,Municipio,Sigla_UF,Regiao,Mes_Ano,Vitimas
0,Bujari,AC,NORTE,2018-01-01,1
1,Cruzeiro do Sul,AC,NORTE,2018-01-01,8
2,Feijó,AC,NORTE,2018-01-01,3
3,Jordão,AC,NORTE,2018-01-01,1
4,Mâncio Lima,AC,NORTE,2018-01-01,1


#### Get cities CEP's

In [130]:
all_ceps = pd.read_excel("C:/Users/User/Documents/data_sets/Lista_de_CEPs.xlsx") 
#Data source: https://terminaldeinformacao.com/2019/01/12/tabela-com-lista-de-ceps-do-brasil/

all_ceps = all_ceps[['Estado', 'Localidade', 'CEP Inicial']]
all_ceps = all_ceps.rename(columns={'Estado':'Sigla_UF', 'Localidade':'Municipio', 'CEP Inicial':'CEP_Inicial'})
all_ceps['Municipio'] = all_ceps['Municipio'].apply(str.lower)
all_ceps['Municipio'] = all_ceps['Municipio'].apply(unidecode)
#all_ceps['Municipio'] = all_ceps['Municipio'].apply(str.replace(' ', ''))
all_ceps['Sigla_UF'] = all_ceps['Sigla_UF'].apply(str.upper)
all_ceps['Sigla_UF'] = all_ceps['Sigla_UF'].apply(unidecode)
#all_ceps['Sigla_UF'] = all_ceps['Sigla_UF'].apply(str.replace(' ', ''))
all_ceps

#A DataFrame containing all cities ceps

,Sigla_UF,Municipio,CEP_Inicial
0,AC,acrelandia,69945000
1,AC,assis brasil,69935000
2,AC,brasileia,69932000
3,AC,bujari,69926000
4,AC,capixaba,69931000
...,...,...,...
6073,SE,gracho cardoso,49860000
6074,SE,n/i,49100000
6075,TO,sao valerio,77390000
6076,TO,tabocao,77708000


In [131]:
def get_cep_by_city_uf(city, uf, cep_table):
    city = unidecode(city.lower())
    uf = unidecode(uf.upper())
    
    ##Option 1 of cross filtering data and returning index - Has been proof to be more effective than the next one
    cep_table = cep_table[(cep_table['Municipio'] == city) & (cep_table['Sigla_UF'] == uf)]
    if len(cep_table) >0:
        return cep_table.iloc[0,2]
    else:
        return 'NaN'
    
    ##Option 2
    '''
    for i, row in cep_table.iterrows():
        if((row['Municipio'].lower() == city.lower()) and (row['Sigla_UF'].upper() == uf.upper())):
            return row['CEP_Inicial']
    
    return 'NaN'
    '''

In [132]:
df['CEP'] = df.progress_apply(lambda x: get_cep_by_city_uf(x['Municipio'], x['Sigla_UF'], all_ceps), axis=1)

100%|██████████| 280085/280085 [19:31<00:00, 239.15it/s]


In [133]:
df['CEP'] = df['CEP'].astype('str')

In [139]:
df.to_csv('C:/Users/User/Documents/data_sets/Brazil_crimes/df_final.csv')